# Predicción de Ventas usando XGBoost y Características Temporales

Este notebook implementa un modelo de predicción de ventas utilizando XGBoost y características temporales avanzadas. El enfoque está basado en el ejemplo de predicción de uso de bicicletas compartidas, pero adaptado al contexto de predicción de ventas en supermercados de Ecuador.

## Objetivos
1. Implementar un modelo XGBoost para predicción de ventas
2. Crear características temporales relevantes
3. Optimizar los hiperparámetros del modelo
4. Evaluar el rendimiento en diferentes condiciones temporales

## Contenido
1. Carga y Preprocesamiento de Datos
2. Ingeniería de Características
3. Configuración del Modelo XGBoost
4. Entrenamiento y Validación
5. Predicción y Evaluación
6. Validación Cruzada Temporal
7. Optimización de Hiperparámetros

In [ ]:
# Data processing
import numpy as np
import pandas as pd
from datetime import datetime

try:
    import feature_engine
    from feature_engine.datetime import DatetimeFeatures
    from feature_engine.creation import CyclicalFeatures
    print("feature_engine imported successfully")
except ImportError as e:
    print(f"Error importing feature_engine: {e}")

try:
    import skforecast
    from skforecast.preprocessing import RollingFeatures
    print("skforecast imported successfully")
except ImportError as e:
    print(f"Error importing skforecast: {e}")

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# Modelling and Forecasting
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
import optuna

# Configuración de warnings
import warnings
warnings.filterwarnings('once')

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("\nVersiones de paquetes instalados:")
print(f"Versión xgboost: {xgb.__version__}")
print(f"Versión numpy: {np.__version__}")
print(f"Versión pandas: {pd.__version__}")
try:
    print(f"Versión feature_engine: {feature_engine.__version__}")
except NameError:
    print("feature_engine no está instalado")
try:
    print(f"Versión skforecast: {skforecast.__version__}")
except NameError:
    print("skforecast no está instalado")

ModuleNotFoundError: No module named 'feature_engine'

## 1. Carga y Preprocesamiento de Datos

En esta sección cargaremos los datos de ventas y realizaremos el preprocesamiento inicial:
1. Carga de archivos CSV
2. Conversión de fechas
3. Manejo de valores faltantes
4. División en conjuntos de entrenamiento, validación y prueba

In [ ]:
# Cargar datos
df_data = pd.read_csv('data/data.csv')
df_stores = pd.read_csv('data/stores.csv')
df_oil = pd.read_csv('data/oil.csv')

# Convertir fechas
df_data['date'] = pd.to_datetime(df_data['date'])
df_oil['date'] = pd.to_datetime(df_oil['date'])

# Mostrar información de los datasets
print("=== Dataset de Ventas ===")
print(df_data.info())
print("\nPrimeras 5 filas:")
print(df_data.head())

print("\n=== Dataset de Tiendas ===")
print(df_stores.info())
print("\nPrimeras 5 filas:")
print(df_stores.head())

print("\n=== Dataset de Precios del Petróleo ===")
print(df_oil.info())
print("\nPrimeras 5 filas:")
print(df_oil.head())

In [ ]:
# Fusionar datos de ventas con información de tiendas
df = df_data.merge(df_stores, on='store_nbr', how='left')

# Fusionar con datos de petróleo
df = df.merge(df_oil, on='date', how='left')

# Manejar valores faltantes
df['dcoilwtico'].fillna(method='ffill', inplace=True)  # Forward fill para precios del petróleo

# Crear índice temporal
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

# Dividir en conjuntos de entrenamiento, validación y test
train_end = '2016-12-31'
valid_end = '2017-04-30'

train_data = df[:'2016-12-31']
valid_data = df['2017-01-01':'2017-04-30']
test_data = df['2017-05-01':]

print("=== División de datos ===")
print(f"Entrenamiento: {train_data.index.min()} hasta {train_data.index.max()} (n={len(train_data)})")
print(f"Validación   : {valid_data.index.min()} hasta {valid_data.index.max()} (n={len(valid_data)})")
print(f"Test         : {test_data.index.min()} hasta {test_data.index.max()} (n={len(test_data)})")

# Visualizar la distribución de ventas
plt.figure(figsize=(12, 6))
plt.hist(df['sales'], bins=50)
plt.title('Distribución de Ventas')
plt.xlabel('Ventas')
plt.ylabel('Frecuencia')
plt.show()

# Estadísticas descriptivas de ventas
print("\n=== Estadísticas de Ventas ===")
print(df['sales'].describe())

## 2. Ingeniería de Características

Crearemos características relevantes para la predicción de ventas:
1. Características temporales (día de la semana, mes, año, etc.)
2. Características cíclicas
3. Características de rezago (lag features)
4. Características de ventana móvil (rolling features)
5. Codificación de variables categóricas

In [ ]:
# Función para crear características
def create_features(df):
    # Características temporales
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['day_of_week'] = df.index.dayofweek
    df['week_of_year'] = df.index.isocalendar().week
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    # Características cíclicas para mes y día de la semana
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)
    
    # One-hot encoding para variables categóricas
    df = pd.get_dummies(df, columns=['type', 'city', 'state'], drop_first=True)
    
    return df

# Función para crear características de lag y ventana móvil
def create_lag_features(df, target='sales', lags=[7, 14, 30], windows=[7, 14, 30]):
    group_cols = ['store_nbr', 'family']
    
    for lag in lags:
        df[f'lag_{lag}'] = df.groupby(group_cols)[target].shift(lag)
    
    for window in windows:
        df[f'rolling_mean_{window}'] = df.groupby(group_cols)[target].transform(
            lambda x: x.shift(1).rolling(window=window).mean()
        )
        df[f'rolling_std_{window}'] = df.groupby(group_cols)[target].transform(
            lambda x: x.shift(1).rolling(window=window).std()
        )
    
    return df

# Aplicar creación de características
print("Creando características...")
df = create_features(df)
df = create_lag_features(df)

# Eliminar filas con valores NaN (primeras filas debido a los lags)
df = df.dropna()

# Mostrar las nuevas características
print("\nNuevas características creadas:")
print(df.columns.tolist())

# Mostrar ejemplo de datos con las nuevas características
print("\nEjemplo de datos con nuevas características:")
print(df.head())

## 3. Configuración del Modelo XGBoost

Configuraremos el modelo XGBoost con parámetros iniciales adecuados para series temporales. 
Los parámetros clave incluyen:
- `max_depth`: Profundidad máxima de los árboles
- `learning_rate`: Tasa de aprendizaje
- `n_estimators`: Número de árboles
- `subsample`: Fracción de muestras para entrenar cada árbol
- `colsample_bytree`: Fracción de características para cada árbol

In [ ]:
# Separar características y objetivo
feature_cols = [col for col in df.columns if col not in ['sales']]
X = df[feature_cols]
y = df['sales']

# Dividir en conjuntos de entrenamiento, validación y test manteniendo el orden temporal
X_train = X[:'2016-12-31']
y_train = y[:'2016-12-31']
X_valid = X['2017-01-01':'2017-04-30']
y_valid = y['2017-01-01':'2017-04-30']
X_test = X['2017-05-01':]
y_test = y['2017-05-01':]

# Configurar modelo XGBoost inicial
xgb_params = {
    'max_depth': 8,
    'learning_rate': 0.1,
    'n_estimators': 1000,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'random_state': 42,
    'tree_method': 'hist',  # Para mejor rendimiento
    'objective': 'reg:squarederror',
}

# Crear el modelo
model = XGBRegressor(**xgb_params)

# Entrenar con early stopping
print("Entrenando modelo XGBoost...")
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_metric=['rmse'],
    early_stopping_rounds=50,
    verbose=100
)

# Evaluar en conjunto de validación
valid_preds = model.predict(X_valid)
valid_rmse = np.sqrt(mean_squared_error(y_valid, valid_preds))
valid_mae = mean_absolute_error(y_valid, valid_preds)

print("\nResultados en conjunto de validación:")
print(f"RMSE: {valid_rmse:.2f}")
print(f"MAE: {valid_mae:.2f}")

# Visualizar importancia de características
plt.figure(figsize=(12, 6))
importances = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
})
importances = importances.sort_values('importance', ascending=False).head(20)
plt.barh(importances['feature'], importances['importance'])
plt.title('Top 20 Características más Importantes')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

## 4. Optimización de Hiperparámetros

Utilizaremos Optuna para realizar una búsqueda eficiente de los mejores hiperparámetros para nuestro modelo XGBoost. 
La optimización se centrará en los siguientes parámetros:
- Profundidad máxima del árbol
- Tasa de aprendizaje
- Número de estimadores
- Submuestreo
- Regularización
- Peso mínimo por hoja

In [ ]:
# Definir función objetivo para Optuna
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'random_state': 42,
        'tree_method': 'hist',
        'objective': 'reg:squarederror'
    }
    
    model = XGBRegressor(**params)
    
    # Entrenar con early stopping
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Predecir en conjunto de validación
    valid_preds = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, valid_preds))
    
    return rmse

# Crear estudio de Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Mostrar mejores parámetros
print("\nMejores parámetros encontrados:")
print(study.best_params)
print(f"\nMejor RMSE: {study.best_value:.2f}")

# Crear y entrenar modelo final con los mejores parámetros
best_params = study.best_params
best_model = XGBRegressor(**best_params, random_state=42, tree_method='hist')

print("\nEntrenando modelo final con los mejores parámetros...")
best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_metric=['rmse'],
    early_stopping_rounds=50,
    verbose=100
)

# Evaluar en conjunto de test
test_preds = best_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
test_mae = mean_absolute_error(y_test, test_preds)

print("\nResultados finales en conjunto de test:")
print(f"RMSE: {test_rmse:.2f}")
print(f"MAE: {test_mae:.2f}")

# Visualizar predicciones vs valores reales
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test.values, label='Real', alpha=0.5)
plt.plot(y_test.index, test_preds, label='Predicción', alpha=0.5)
plt.title('Predicciones vs Valores Reales en Conjunto de Test')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.tight_layout()
plt.show()